# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,24.18,83,98,4.36,PN,1707188628
1,1,portland,45.5234,-122.6762,9.49,93,100,3.60,US,1707187901
2,2,petropavlovsk-kamchatsky,53.0452,158.6483,-5.67,74,75,2.68,RU,1707188032
3,3,tamanrasset,22.7850,5.5228,8.95,27,0,6.17,DZ,1707188050
4,4,ust-nera,64.5667,143.2000,-25.02,86,99,0.71,RU,1707188041


In [3]:
"https://api.openweathermap.org/data/2.5/weather?lat=57&lon=-2.15&appid={weather_api_key}&units=imperial"

'https://api.openweathermap.org/data/2.5/weather?lat=57&lon=-2.15&appid={weather_api_key}&units=imperial'

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng","Lat",geo=True,tiles ="OSM")

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_condition = city_data_df.loc[(city_data_df["Max Temp"] <= 90) & (city_data_df["Max Temp"] >= 70)]
ideal_condition = city_data_df.loc[city_data_df["Wind Speed"] < 10]
ideal_condition = city_data_df.loc[city_data_df["Cloudiness"] == 0]
# Drop any rows with null values
ideal_weather_df = ideal_condition.dropna()
# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,tamanrasset,22.7850,5.5228,8.95,27,0,6.17,DZ,1707188050
6,6,ushuaia,-54.8000,-68.3000,9.81,62,0,1.03,AR,1707188630
8,8,edinburgh of the seven seas,-37.0676,-12.3116,15.61,63,0,1.87,SH,1707188611
46,46,bethel,41.3712,-73.4140,0.52,53,0,0.45,US,1707188604
57,57,albany,42.6001,-73.9662,-2.55,56,0,0.89,US,1707188219
...,...,...,...,...,...,...,...,...,...,...
554,554,vila do maio,15.1333,-23.2167,22.44,62,0,6.36,CV,1707188806
560,560,santa cruz del sur,20.7147,-77.9956,23.90,68,0,11.82,CU,1707188777
563,563,greenville,35.6127,-77.3663,5.64,71,0,3.60,US,1707188809
566,566,freetown,8.4840,-13.2299,23.81,82,0,1.68,SL,1707188810


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City","Lat", "Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Country","Date"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,tamanrasset,22.7850,5.5228,8.95,27,0,6.17,DZ,1707188050,
6,ushuaia,-54.8000,-68.3000,9.81,62,0,1.03,AR,1707188630,
8,edinburgh of the seven seas,-37.0676,-12.3116,15.61,63,0,1.87,SH,1707188611,
46,bethel,41.3712,-73.4140,0.52,53,0,0.45,US,1707188604,
57,albany,42.6001,-73.9662,-2.55,56,0,0.89,US,1707188219,
90,pisco,-13.7000,-76.2167,25.03,83,0,5.14,PE,1707188657,
98,jamestown,42.0970,-79.2353,-2.52,94,0,4.12,US,1707188661,
103,sittwe,20.1500,92.9000,23.30,47,0,2.07,MM,1707188181,
107,nova sintra,14.8667,-24.7167,20.87,68,0,6.12,CV,1707188664,
110,swan hill,-35.3378,143.5544,25.49,20,0,7.06,AU,1707188664,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "radius": 5000,
    "limit": 20,
    "key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
   
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tamanrasset - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
sittwe - nearest hotel: No hotel found
nova sintra - nearest hotel: No hotel found
swan hill - nearest hotel: No hotel found
vanino - nearest hotel: No hotel found
timbuktu - nearest hotel: No hotel found
port lincoln - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
queenstown - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
dawson creek - nearest hotel: No hotel found
mount gambier - nearest hotel: No hotel found
puerto real - nearest hotel: No hotel found
aleg - nearest hotel: No hotel found
valparaiso - nearest hotel: No hotel found
cidade velha - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: No hot

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,tamanrasset,22.7850,5.5228,8.95,27,0,6.17,DZ,1707188050,No hotel found
6,ushuaia,-54.8000,-68.3000,9.81,62,0,1.03,AR,1707188630,No hotel found
8,edinburgh of the seven seas,-37.0676,-12.3116,15.61,63,0,1.87,SH,1707188611,No hotel found
46,bethel,41.3712,-73.4140,0.52,53,0,0.45,US,1707188604,No hotel found
57,albany,42.6001,-73.9662,-2.55,56,0,0.89,US,1707188219,No hotel found
...,...,...,...,...,...,...,...,...,...,...
554,vila do maio,15.1333,-23.2167,22.44,62,0,6.36,CV,1707188806,No hotel found
560,santa cruz del sur,20.7147,-77.9956,23.90,68,0,11.82,CU,1707188777,No hotel found
563,greenville,35.6127,-77.3663,5.64,71,0,3.60,US,1707188809,No hotel found
566,freetown,8.4840,-13.2299,23.81,82,0,1.68,SL,1707188810,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng","Lat",geo=True,tiles ="Hotel Map")

# Display the map
hotel_display()

ValueError: Hotel Map tiles not recognized, must be one of: ['CartoDark', 'CartoEco', 'CartoLight', 'CartoMidnight', 'EsriAntarcticImagery', 'EsriArcticImagery', 'EsriArcticOceanBase', 'EsriArcticOceanReference', 'EsriDelormeWorldBaseMap', 'EsriImagery', 'EsriNatGeo', 'EsriOceanBase', 'EsriOceanReference', 'EsriReference', 'EsriTerrain', 'EsriUSATopo', 'EsriWorldBoundariesAndPlaces', 'EsriWorldBoundariesAndPlacesAlternate', 'EsriWorldDarkGrayBase', 'EsriWorldDarkGrayReference', 'EsriWorldHillshade', 'EsriWorldHillshadeDark', 'EsriWorldLightGrayBase', 'EsriWorldLightGrayReference', 'EsriWorldNavigationCharts', 'EsriWorldPhysical', 'EsriWorldShadedRelief', 'EsriWorldStreetMap', 'EsriWorldTopo', 'EsriWorldTransportation', 'OSM', 'OpenTopoMap'] or a tile object